# Ciência de Dados 2

### Limpeza dos dados do Twitter

##### fonte: https://towardsdatascience.com/preprocessing-text-data-using-python-576206753c28

In [81]:
import pandas as pd
import numpy as np
import joblib
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('mac_morpho')
# nltk.download('rslp')
# nltk.download('omw')
# nltk.download('floresta')

import string
import fasttext
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem.snowball import SnowballStemmer
import unicodedata
import re

pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

ModuleNotFoundError: No module named 'snowballstemmer'

In [70]:
# lê o arquivo CSV
df = pd.read_csv('datasets/PrefsTweetsAgregado.csv', delimiter=";")

# mostra o conteúdo do DataFrame
df.head()

,Unnamed: 0,municipio,tweet_id,created_at,author_id,text,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count,referenced_twitter
0,0,São Paulo,1380999403910094855,2021-04-10T21:41:30.000Z,1319636844821217280,"Esse da foto é o Pluto, um filhote calminho e super brincalhão. Ele e seus irmãos estão a espera...",4,0,9,0,False
1,1,São Paulo,1380985859059576832,2021-04-10T20:47:40.000Z,1319636844821217280,O início da vacinação dos idosos com 67 anos ou mais foi antecipado para segunda (12). A estimat...,0,1,1,0,False
2,2,São Paulo,1380969544513159176,2021-04-10T19:42:51.000Z,1319636844821217280,RT @spsmdet: Uma onda de solidariedade chegou ao Banco de Alimentos!\n\nNo primeiro semestre de ...,2,0,0,0,True
3,3,São Paulo,1380960415555915781,2021-04-10T19:06:34.000Z,1319636844821217280,A Secretaria de Educação divulgou a convocação de 55 candidatos aprovados no concurso público pa...,0,0,2,0,False
4,4,São Paulo,1380953886299713541,2021-04-10T18:40:38.000Z,1319636844821217280,Os postos drive-thru que fazem parte da estrutura da campanha de vacinação contra a Covid-19 na ...,1,0,3,0,False


### Checando os dados verificamos que todos os tweets possuem o author_id da propria prefeitura, quando o tweet e uma resposta ou é retweetado entao temos a ultima coluna para confirmar. Por enquanto decido dropar a coluna author_id. A outra coluna foi resultado da agregação dos dados do Twitter então remove ela também. 

In [71]:
df.drop('author_id', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167919 entries, 0 to 167918
Data columns (total 9 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   municipio                     167919 non-null  object
 1   tweet_id                      167919 non-null  int64 
 2   created_at                    167919 non-null  object
 3   text                          167919 non-null  object
 4   public_metrics_retweet_count  167919 non-null  int64 
 5   public_metrics_reply_count    167919 non-null  int64 
 6   public_metrics_like_count     167919 non-null  int64 
 7   public_metrics_quote_count    167919 non-null  int64 
 8   referenced_twitter            167919 non-null  bool  
dtypes: bool(1), int64(5), object(3)
memory usage: 10.4+ MB


### Observando o tutorial que esta na fonte, tem uma parte com contrações e idioma devido ao tutorial ser para a língua inglesa, então decido pular essa parte pois todas as prefeituras são do Brasil.

### Removo também os acentos do texto do tweet.

In [73]:
def remove_punctuations(text):
    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', text)
    wordNew = u"".join([c for c in nfkd if not unicodedata.combining(c)])

    return re.sub('[^a-zA-Z0-9 \\\]', '', wordNew)

df['text'] = df['text'].apply(remove_punctuations)
df.head()

,municipio,tweet_id,created_at,text,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count,referenced_twitter
0,São Paulo,1380999403910094855,2021-04-10T21:41:30.000Z,Esse da foto e o Pluto um filhote calminho e super brincalhao Ele e seus irmaos estao a espera d...,4,0,9,0,False
1,São Paulo,1380985859059576832,2021-04-10T20:47:40.000Z,O inicio da vacinacao dos idosos com 67 anos ou mais foi antecipado para segunda 12 A estimativa...,0,1,1,0,False
2,São Paulo,1380969544513159176,2021-04-10T19:42:51.000Z,RT spsmdet Uma onda de solidariedade chegou ao Banco de AlimentosNo primeiro semestre de 2021 fo...,2,0,0,0,True
3,São Paulo,1380960415555915781,2021-04-10T19:06:34.000Z,A Secretaria de Educacao divulgou a convocacao de 55 candidatos aprovados no concurso publico pa...,0,0,2,0,False
4,São Paulo,1380953886299713541,2021-04-10T18:40:38.000Z,Os postos drivethru que fazem parte da estrutura da campanha de vacinacao contra a Covid19 na ci...,1,0,3,0,False


### Agora faco a tokenizacao 

In [74]:
df['tokens'] = df['text'].apply(word_tokenize, language='portuguese')

df = df[['municipio', 'tweet_id', 'created_at', 'text', 'tokens', 'referenced_twitter', 'public_metrics_retweet_count', 'public_metrics_reply_count', 'public_metrics_like_count', 'public_metrics_quote_count' ]]

df.head()

,municipio,tweet_id,created_at,text,tokens,referenced_twitter,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count
0,São Paulo,1380999403910094855,2021-04-10T21:41:30.000Z,Esse da foto e o Pluto um filhote calminho e super brincalhao Ele e seus irmaos estao a espera d...,"[Esse, da, foto, e, o, Pluto, um, filhote, calminho, e, super, brincalhao, Ele, e, seus, irmaos,...",False,4,0,9,0
1,São Paulo,1380985859059576832,2021-04-10T20:47:40.000Z,O inicio da vacinacao dos idosos com 67 anos ou mais foi antecipado para segunda 12 A estimativa...,"[O, inicio, da, vacinacao, dos, idosos, com, 67, anos, ou, mais, foi, antecipado, para, segunda,...",False,0,1,1,0
2,São Paulo,1380969544513159176,2021-04-10T19:42:51.000Z,RT spsmdet Uma onda de solidariedade chegou ao Banco de AlimentosNo primeiro semestre de 2021 fo...,"[RT, spsmdet, Uma, onda, de, solidariedade, chegou, ao, Banco, de, AlimentosNo, primeiro, semest...",True,2,0,0,0
3,São Paulo,1380960415555915781,2021-04-10T19:06:34.000Z,A Secretaria de Educacao divulgou a convocacao de 55 candidatos aprovados no concurso publico pa...,"[A, Secretaria, de, Educacao, divulgou, a, convocacao, de, 55, candidatos, aprovados, no, concur...",False,0,0,2,0
4,São Paulo,1380953886299713541,2021-04-10T18:40:38.000Z,Os postos drivethru que fazem parte da estrutura da campanha de vacinacao contra a Covid19 na ci...,"[Os, postos, drivethru, que, fazem, parte, da, estrutura, da, campanha, de, vacinacao, contra, a...",False,1,0,3,0


### Agora com a parte de texto pronta, vou aplicar algumas modificações para limpeza. A primeira sera deixar tudo em minusculo. 

In [75]:
df['tokens'] = df['tokens'].apply(lambda x: [word.lower() for word in x])

df.head()

,municipio,tweet_id,created_at,text,tokens,referenced_twitter,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count
0,São Paulo,1380999403910094855,2021-04-10T21:41:30.000Z,Esse da foto e o Pluto um filhote calminho e super brincalhao Ele e seus irmaos estao a espera d...,"[esse, da, foto, e, o, pluto, um, filhote, calminho, e, super, brincalhao, ele, e, seus, irmaos,...",False,4,0,9,0
1,São Paulo,1380985859059576832,2021-04-10T20:47:40.000Z,O inicio da vacinacao dos idosos com 67 anos ou mais foi antecipado para segunda 12 A estimativa...,"[o, inicio, da, vacinacao, dos, idosos, com, 67, anos, ou, mais, foi, antecipado, para, segunda,...",False,0,1,1,0
2,São Paulo,1380969544513159176,2021-04-10T19:42:51.000Z,RT spsmdet Uma onda de solidariedade chegou ao Banco de AlimentosNo primeiro semestre de 2021 fo...,"[rt, spsmdet, uma, onda, de, solidariedade, chegou, ao, banco, de, alimentosno, primeiro, semest...",True,2,0,0,0
3,São Paulo,1380960415555915781,2021-04-10T19:06:34.000Z,A Secretaria de Educacao divulgou a convocacao de 55 candidatos aprovados no concurso publico pa...,"[a, secretaria, de, educacao, divulgou, a, convocacao, de, 55, candidatos, aprovados, no, concur...",False,0,0,2,0
4,São Paulo,1380953886299713541,2021-04-10T18:40:38.000Z,Os postos drivethru que fazem parte da estrutura da campanha de vacinacao contra a Covid19 na ci...,"[os, postos, drivethru, que, fazem, parte, da, estrutura, da, campanha, de, vacinacao, contra, a...",False,1,0,3,0


### Agora retiro parte da pontuacao e depois removo os acentos. 

In [76]:
punc = string.punctuation
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in punc])

df.head()

,municipio,tweet_id,created_at,text,tokens,referenced_twitter,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count
0,São Paulo,1380999403910094855,2021-04-10T21:41:30.000Z,Esse da foto e o Pluto um filhote calminho e super brincalhao Ele e seus irmaos estao a espera d...,"[esse, da, foto, e, o, pluto, um, filhote, calminho, e, super, brincalhao, ele, e, seus, irmaos,...",False,4,0,9,0
1,São Paulo,1380985859059576832,2021-04-10T20:47:40.000Z,O inicio da vacinacao dos idosos com 67 anos ou mais foi antecipado para segunda 12 A estimativa...,"[o, inicio, da, vacinacao, dos, idosos, com, 67, anos, ou, mais, foi, antecipado, para, segunda,...",False,0,1,1,0
2,São Paulo,1380969544513159176,2021-04-10T19:42:51.000Z,RT spsmdet Uma onda de solidariedade chegou ao Banco de AlimentosNo primeiro semestre de 2021 fo...,"[rt, spsmdet, uma, onda, de, solidariedade, chegou, ao, banco, de, alimentosno, primeiro, semest...",True,2,0,0,0
3,São Paulo,1380960415555915781,2021-04-10T19:06:34.000Z,A Secretaria de Educacao divulgou a convocacao de 55 candidatos aprovados no concurso publico pa...,"[a, secretaria, de, educacao, divulgou, a, convocacao, de, 55, candidatos, aprovados, no, concur...",False,0,0,2,0
4,São Paulo,1380953886299713541,2021-04-10T18:40:38.000Z,Os postos drivethru que fazem parte da estrutura da campanha de vacinacao contra a Covid19 na ci...,"[os, postos, drivethru, que, fazem, parte, da, estrutura, da, campanha, de, vacinacao, contra, a...",False,1,0,3,0


In [77]:
## Agora retiro as stop words

stop_words = set(stopwords.words('portuguese'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

df.head()

,municipio,tweet_id,created_at,text,tokens,referenced_twitter,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count
0,São Paulo,1380999403910094855,2021-04-10T21:41:30.000Z,Esse da foto e o Pluto um filhote calminho e super brincalhao Ele e seus irmaos estao a espera d...,"[foto, pluto, filhote, calminho, super, brincalhao, irmaos, estao, espera, familia, cheia, amor,...",False,4,0,9,0
1,São Paulo,1380985859059576832,2021-04-10T20:47:40.000Z,O inicio da vacinacao dos idosos com 67 anos ou mais foi antecipado para segunda 12 A estimativa...,"[inicio, vacinacao, idosos, 67, anos, antecipado, segunda, 12, estimativa, 96760, pessoas, nessa...",False,0,1,1,0
2,São Paulo,1380969544513159176,2021-04-10T19:42:51.000Z,RT spsmdet Uma onda de solidariedade chegou ao Banco de AlimentosNo primeiro semestre de 2021 fo...,"[rt, spsmdet, onda, solidariedade, chegou, banco, alimentosno, primeiro, semestre, 2021, arrecad...",True,2,0,0,0
3,São Paulo,1380960415555915781,2021-04-10T19:06:34.000Z,A Secretaria de Educacao divulgou a convocacao de 55 candidatos aprovados no concurso publico pa...,"[secretaria, educacao, divulgou, convocacao, 55, candidatos, aprovados, concurso, publico, profe...",False,0,0,2,0
4,São Paulo,1380953886299713541,2021-04-10T18:40:38.000Z,Os postos drivethru que fazem parte da estrutura da campanha de vacinacao contra a Covid19 na ci...,"[postos, drivethru, fazem, parte, estrutura, campanha, vacinacao, contra, covid19, cidade, sao, ...",False,1,0,3,0


In [78]:
## Agora retiro os links

df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if "http" not in word ])

df.head()

,municipio,tweet_id,created_at,text,tokens,referenced_twitter,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count
0,São Paulo,1380999403910094855,2021-04-10T21:41:30.000Z,Esse da foto e o Pluto um filhote calminho e super brincalhao Ele e seus irmaos estao a espera d...,"[foto, pluto, filhote, calminho, super, brincalhao, irmaos, estao, espera, familia, cheia, amor,...",False,4,0,9,0
1,São Paulo,1380985859059576832,2021-04-10T20:47:40.000Z,O inicio da vacinacao dos idosos com 67 anos ou mais foi antecipado para segunda 12 A estimativa...,"[inicio, vacinacao, idosos, 67, anos, antecipado, segunda, 12, estimativa, 96760, pessoas, nessa...",False,0,1,1,0
2,São Paulo,1380969544513159176,2021-04-10T19:42:51.000Z,RT spsmdet Uma onda de solidariedade chegou ao Banco de AlimentosNo primeiro semestre de 2021 fo...,"[rt, spsmdet, onda, solidariedade, chegou, banco, alimentosno, primeiro, semestre, 2021, arrecad...",True,2,0,0,0
3,São Paulo,1380960415555915781,2021-04-10T19:06:34.000Z,A Secretaria de Educacao divulgou a convocacao de 55 candidatos aprovados no concurso publico pa...,"[secretaria, educacao, divulgou, convocacao, 55, candidatos, aprovados, concurso, publico, profe...",False,0,0,2,0
4,São Paulo,1380953886299713541,2021-04-10T18:40:38.000Z,Os postos drivethru que fazem parte da estrutura da campanha de vacinacao contra a Covid19 na ci...,"[postos, drivethru, fazem, parte, estrutura, campanha, vacinacao, contra, covid19, cidade, sao, ...",False,1,0,3,0


### Agora comeco o processo de stemming. 

#### Update : Fiz um primeiro com com a nltk.tag.pos_tag porem a biblioteca nao e boa para classificacao em portugues, so funciona para o ingles. Entao baixei uma pos_tagger ja treinada para poder finalizar o processo de lematização


#### Udate: Tentei utilizar alguns modelos pre treinados mas nao consegui termnar a tempo entao decido usar stemming que ja esta pronto para portugues


In [79]:
## df['pos_tag'] = df['tokens'].apply(nltk.tag.pos_tag)

## def get_wordnet_pos(tag):
##     if tag.startswith('J'):
##         return wn.ADJ
##     elif tag.startswith('V'):
##         return wn.VERB
##     elif tag.startswith('V'):
##         return wn.VERB
##     elif tag.startswith('N'):
##         return wn.NOUN
##     elif tag.startswith('R'):
##         return wn.ADV
##     elif tag.startswith('R'):
##         return wn.ADV
##     else:
##        return wn.NOUN
    
##  df['wordnet_pos'] = df['pos_tag'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

##  wnl = WordNetLemmatizer()

##  df['final_tokens'] = df['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])

In [91]:
stemmer = SnowballStemmer("portuguese");

df['final_tokens'] = df['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

df.head()

,municipio,tweet_id,created_at,text,tokens,referenced_twitter,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count,final_tokens
0,São Paulo,1380999403910094855,2021-04-10T21:41:30.000Z,Esse da foto e o Pluto um filhote calminho e super brincalhao Ele e seus irmaos estao a espera d...,"[foto, pluto, filhote, calminho, super, brincalhao, irmaos, estao, espera, familia, cheia, amor,...",False,4,0,9,0,"[fot, plut, filhot, calminh, sup, brincalha, irma, esta, esper, famil, che, amor, cosap, adot, p..."
1,São Paulo,1380985859059576832,2021-04-10T20:47:40.000Z,O inicio da vacinacao dos idosos com 67 anos ou mais foi antecipado para segunda 12 A estimativa...,"[inicio, vacinacao, idosos, 67, anos, antecipado, segunda, 12, estimativa, 96760, pessoas, nessa...",False,0,1,1,0,"[inici, vacinaca, idos, 67, anos, antecip, segund, 12, estim, 96760, pesso, ness, faix, etar, re..."
2,São Paulo,1380969544513159176,2021-04-10T19:42:51.000Z,RT spsmdet Uma onda de solidariedade chegou ao Banco de AlimentosNo primeiro semestre de 2021 fo...,"[rt, spsmdet, onda, solidariedade, chegou, banco, alimentosno, primeiro, semestre, 2021, arrecad...",True,2,0,0,0,"[rt, spsmdet, onda, solidariedad, cheg, banc, alimentosn, primeir, semestr, 2021, arrecad, 730, ..."
3,São Paulo,1380960415555915781,2021-04-10T19:06:34.000Z,A Secretaria de Educacao divulgou a convocacao de 55 candidatos aprovados no concurso publico pa...,"[secretaria, educacao, divulgou, convocacao, 55, candidatos, aprovados, concurso, publico, profe...",False,0,0,2,0,"[secret, educaca, divulg, convocaca, 55, candidat, aprov, concurs, public, professor, educaca, i..."
4,São Paulo,1380953886299713541,2021-04-10T18:40:38.000Z,Os postos drivethru que fazem parte da estrutura da campanha de vacinacao contra a Covid19 na ci...,"[postos, drivethru, fazem, parte, estrutura, campanha, vacinacao, contra, covid19, cidade, sao, ...",False,1,0,3,0,"[post, drivethru, faz, part, estrutur, campanh, vacinaca, contr, covid19, cidad, sao, paul, esta..."


In [92]:
df.to_csv('datasets/PrefsTweetsLimpoStemmerSB.csv')

df.head()

,municipio,tweet_id,created_at,text,tokens,referenced_twitter,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count,final_tokens
0,São Paulo,1380999403910094855,2021-04-10T21:41:30.000Z,Esse da foto e o Pluto um filhote calminho e super brincalhao Ele e seus irmaos estao a espera d...,"[foto, pluto, filhote, calminho, super, brincalhao, irmaos, estao, espera, familia, cheia, amor,...",False,4,0,9,0,"[fot, plut, filhot, calminh, sup, brincalha, irma, esta, esper, famil, che, amor, cosap, adot, p..."
1,São Paulo,1380985859059576832,2021-04-10T20:47:40.000Z,O inicio da vacinacao dos idosos com 67 anos ou mais foi antecipado para segunda 12 A estimativa...,"[inicio, vacinacao, idosos, 67, anos, antecipado, segunda, 12, estimativa, 96760, pessoas, nessa...",False,0,1,1,0,"[inici, vacinaca, idos, 67, anos, antecip, segund, 12, estim, 96760, pesso, ness, faix, etar, re..."
2,São Paulo,1380969544513159176,2021-04-10T19:42:51.000Z,RT spsmdet Uma onda de solidariedade chegou ao Banco de AlimentosNo primeiro semestre de 2021 fo...,"[rt, spsmdet, onda, solidariedade, chegou, banco, alimentosno, primeiro, semestre, 2021, arrecad...",True,2,0,0,0,"[rt, spsmdet, onda, solidariedad, cheg, banc, alimentosn, primeir, semestr, 2021, arrecad, 730, ..."
3,São Paulo,1380960415555915781,2021-04-10T19:06:34.000Z,A Secretaria de Educacao divulgou a convocacao de 55 candidatos aprovados no concurso publico pa...,"[secretaria, educacao, divulgou, convocacao, 55, candidatos, aprovados, concurso, publico, profe...",False,0,0,2,0,"[secret, educaca, divulg, convocaca, 55, candidat, aprov, concurs, public, professor, educaca, i..."
4,São Paulo,1380953886299713541,2021-04-10T18:40:38.000Z,Os postos drivethru que fazem parte da estrutura da campanha de vacinacao contra a Covid19 na ci...,"[postos, drivethru, fazem, parte, estrutura, campanha, vacinacao, contra, covid19, cidade, sao, ...",False,1,0,3,0,"[post, drivethru, faz, part, estrutur, campanh, vacinaca, contr, covid19, cidad, sao, paul, esta..."


### Salvo a versao final dos tweets ja pre-processados e prontos para a analise